In [1]:
# vamos a hacer web scrpping para obtener datos deuna tabla de wikipedia sobre conflictos armados
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [107]:
import numpy as np

In [42]:
url = 'https://wisevoter.com/country-rankings/countries-currently-at-war/#russia'

In [43]:
response = requests.get(url, 'html.parser')
print(response)

<Response [200]>


In [44]:
soup = BeautifulSoup(response.content)
soup

<!DOCTYPE html>
<html class="no-js" lang="en-US"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><link href="https://gmpg.org/xfn/11" rel="profile"/><meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/><title>Countries Currently at War 2023 - Wisevoter</title><link href="https://wisevoter.com/country-rankings/countries-currently-at-war/" rel="canonical"/><meta content="en_US" property="og:locale"/><meta content="article" property="og:type"/><meta content="Countries Currently at War 2023 - Wisevoter" property="og:title"/><meta content="Overall, there are 32 countries currently in conflict, and the types of conflict vary widely. While the severity and duration of these conflicts differ, they all have significant impacts on the affected populations and can result in a high number of casualties as well as humanitarian crises. One of the most common types of conflict […]" property="og:

In [45]:
tabla = soup.find('table', {'class': 'shdb-on-page-table', 'id': 'shdb-on-page-table'})
tabla

<table class="shdb-on-page-table" id="shdb-on-page-table"><thead><tr><th>Country</th><th>Conflict Type</th><th>Conflict Casualties</th></tr></thead><tbody><tr><td class="shdb-on-page-table-body-Geo">Afghanistan</td><td class="shdb-on-page-table-body-Data" data-order="1">Civil War</td><td class="shdb-on-page-table-body-Data"> 1,000 to 10,000</td></tr><tr><td class="shdb-on-page-table-body-Geo">Central African Republic</td><td class="shdb-on-page-table-body-Data" data-order="2">Civil War</td><td class="shdb-on-page-table-body-Data"> 0 to 999</td></tr><tr><td class="shdb-on-page-table-body-Geo">Ethiopia</td><td class="shdb-on-page-table-body-Data" data-order="3">Civil War</td><td class="shdb-on-page-table-body-Data"> 1,000 to 10,000</td></tr><tr><td class="shdb-on-page-table-body-Geo">Libya</td><td class="shdb-on-page-table-body-Data" data-order="4">Civil War</td><td class="shdb-on-page-table-body-Data"> 0 to 999</td></tr><tr><td class="shdb-on-page-table-body-Geo">Mali</td><td class="shd

In [46]:
columnas = tabla.find_all('th')
columnas

[<th>Country</th>, <th>Conflict Type</th>, <th>Conflict Casualties</th>]

In [47]:
columnas = [columna.text.strip() for columna in columnas]

In [48]:
columnas

['Country', 'Conflict Type', 'Conflict Casualties']

In [49]:
df = pd.DataFrame(columns=columnas)
df

,Country,Conflict Type,Conflict Casualties


In [69]:
rows = soup.find_all('td')
rows

[<td class="shdb-on-page-table-body-Geo">Afghanistan</td>,
 <td class="shdb-on-page-table-body-Data" data-order="1">Civil War</td>,
 <td class="shdb-on-page-table-body-Data"> 1,000 to 10,000</td>,
 <td class="shdb-on-page-table-body-Geo">Central African Republic</td>,
 <td class="shdb-on-page-table-body-Data" data-order="2">Civil War</td>,
 <td class="shdb-on-page-table-body-Data"> 0 to 999</td>,
 <td class="shdb-on-page-table-body-Geo">Ethiopia</td>,
 <td class="shdb-on-page-table-body-Data" data-order="3">Civil War</td>,
 <td class="shdb-on-page-table-body-Data"> 1,000 to 10,000</td>,
 <td class="shdb-on-page-table-body-Geo">Libya</td>,
 <td class="shdb-on-page-table-body-Data" data-order="4">Civil War</td>,
 <td class="shdb-on-page-table-body-Data"> 0 to 999</td>,
 <td class="shdb-on-page-table-body-Geo">Mali</td>,
 <td class="shdb-on-page-table-body-Data" data-order="5">Civil War</td>,
 <td class="shdb-on-page-table-body-Data"> 1,000 to 10,000</td>,
 <td class="shdb-on-page-table-b

In [76]:
rows = soup.find_all('tr')
rows

[<tr><th>Country</th><th>Conflict Type</th><th>Conflict Casualties</th></tr>,
 <tr><td class="shdb-on-page-table-body-Geo">Afghanistan</td><td class="shdb-on-page-table-body-Data" data-order="1">Civil War</td><td class="shdb-on-page-table-body-Data"> 1,000 to 10,000</td></tr>,
 <tr><td class="shdb-on-page-table-body-Geo">Central African Republic</td><td class="shdb-on-page-table-body-Data" data-order="2">Civil War</td><td class="shdb-on-page-table-body-Data"> 0 to 999</td></tr>,
 <tr><td class="shdb-on-page-table-body-Geo">Ethiopia</td><td class="shdb-on-page-table-body-Data" data-order="3">Civil War</td><td class="shdb-on-page-table-body-Data"> 1,000 to 10,000</td></tr>,
 <tr><td class="shdb-on-page-table-body-Geo">Libya</td><td class="shdb-on-page-table-body-Data" data-order="4">Civil War</td><td class="shdb-on-page-table-body-Data"> 0 to 999</td></tr>,
 <tr><td class="shdb-on-page-table-body-Geo">Mali</td><td class="shdb-on-page-table-body-Data" data-order="5">Civil War</td><td clas

In [82]:
# Utilizaremos listas para guardar cada elemento de la fila en una columna
countries = []
conflict_types = []
casualties = []


# Iterar sobre cada fila
for row in rows:
    # Encontrar todas las celdas (td) en la fila
    cells = row.find_all('td')
    
    # Verificar si hay al menos tres celdas antes de intentar acceder a ellas
    if len(cells) >= 3:
        # Guardar los datos en las listas respectivas
        countries.append(cells[0].text.strip())
        conflict_types.append(cells[1].text.strip())
        casualties.append(cells[2].text.strip())

In [148]:
current_war = pd.DataFrame({'Country': countries,'Conflict_type':conflict_types,'Conflict Casualties':casualties})
current_war

,Country,Conflict_type,Conflict Casualties
0,Afghanistan,Civil War,"1,000 to 10,000"
1,Central African Republic,Civil War,0 to 999
2,Ethiopia,Civil War,"1,000 to 10,000"
3,Libya,Civil War,0 to 999
4,Mali,Civil War,"1,000 to 10,000"
5,Myanmar,Civil War,"10,000+"
6,Somalia,Civil War,"1,000 to 10,000"
7,Syria,Civil War,"1,000 to 10,000"
8,Yemen,Civil War,"1,000 to 10,000"
9,Colombia,Drug War,"1,000 to 10,000"


In [149]:
current_war.to_csv('Data/current_war.csv', index=False)